In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train) ,(x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
y_train.shape

(50000, 1)

In [ ]:
x_test.shape # kendisi test için 10k ayırmış

(10000, 32, 32, 3)

In [ ]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [ ]:
classes[4]

'deer'

In [ ]:
index = 900
plt.imshow(x_train[index], vmin = 32, vmax = 32)
print(classes[int(y_train[index])])  # y_train içindeki 900 index'li

In [ ]:
index = 800
plt.imshow(x_train[index], vmin=32, vmax=32) # vmin ve vmax resimin boyutunu belirliyor
print(classes[int(y_train[index])])

In [ ]:
print(x_train.dtype)

uint8


In [ ]:
x_train = x_train.astype("float32") # daha iyi bir sonuç alabilmek için float32 yaptık
x_test = x_test.astype("float32")  # astype tipini değiştiriyor

In [ ]:
print(x_train.dtype)

float32


In [ ]:
x_train

In [ ]:
x_train /= 255
x_test /= 255  # resmimizin piksel değerlerini 0-1 arasına getirdik

In [ ]:
x_train[8] # kontrol ettik 0-1 arasında olmuş mu diye

In [ ]:
x_train.shape # shape 4 oldugu  için değiştirmedik. zaten 4 olması gerekiyordu

(50000, 32, 32, 3)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
my_model = Sequential()  # katmanlar arasında verimiz soldan sağa doğru akıyor
my_model.add(Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu", input_shape=(32,32,3)))
my_model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu",))
my_model.add(MaxPooling2D(pool_size=(2,2)))
my_model.add(Flatten()) #kendine gelen çokboyutlu veriyi vektöre indigiyor. tek boyutlu yapıyor
my_model.add(Dense(64, activation="relu")) # hangi activation fonksiyonunu verirsek o fonksiyonları yerine getiriyor onun dışında başka bir işlem yapmıyor. 64 ise nöron sayısını belirtir.
my_model.add(Dense(10, activation="softmax")) # 10 tane nokta var yani nöron var son katmanda, her nöron atıyorum %20 çıktı 1. nöron için %20 çıktıysa airplane için %20 ihtimal vermiş
# softmax bütün çıktıların toplamını 0-1 arasına sıkıştırıyor. Benim anladığım kadarıyla herbir çıktı nöronuna yüzde ihitmal veriyor yani %20 ihtimaldir falan filan

In [ ]:
my_model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(my_model)

In [ ]:
my_model.compile(loss="categorical_crossentropy", # hatasını nasıl ölçsün hangi hata fonksiyonunu kullanarak hatasını ölçsün o da categorical_crossentropy ile ölçsün
                 optimizer=Adam(), # Adama optimizerine göre kendini iyileştirsin
                 metrics=["accuracy"]) # kendi performansını nasıl baksın. accuracy metrics'i ile baksın. accuracy'im %20 desin ve %20 performans yüklesin(desin). sonrada bu hatayı ölçme yöntemin olsun. accuracy cinsinden vericek
                 # bu hatayı. bu hatayı ne ile ölçsün categorical_crossentropy ile ölçsün. Adam optimizer ile de düzeltsin
                 # accuracy'ye göre performansımı değerlendiricem. loss fonkisyonu ile hatamı ölçüyorum. Adam fonksiyonu ile de hatamı iyileştiriyorum.
                 # bu sürekli yakınsayana kadar devam ediyor.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
?ImageDataGenerator

In [ ]:
data_generator = ImageDataGenerator(
    rotation_range = 360, # resimleri 40 dereceye kadar çeviripçevirip alıyor --- en yüksek accuracy 360 derecede çıktı
    width_shift_range = 1,
    height_shift_range = 1,
    shear_range = 1,
    zoom_range = 1,
    horizontal_flip = True,
    fill_mode = "nearest"
)

In [ ]:
history = my_model.fit(x_train, y_train,
batch_size=5096, # genelde 2 nin üsleri şeklinde verilir --- yaptığım denemelere göre 256 dan yüksek 5096 dan küçük olsa daha iyi sonuçlara verecek
epochs=30, # verilerin kaç defa dönüyor. modeli bir kere dolaştığında epochs 1 olur --- yaptığım deneylere göre epochs'un yüksek olması accuracy'nin artmasını sağlıyor.
verbose=1, # verbose nasıl çıktı ile sunulur veriler bunu göster
validation_split=0.2)

In [ ]:
score = my_model.evaluate(x_test, y_test, verbose=1)
print("test loss:",score[0])
print("test accuracy:",score[1])

313/313 [==============================] - 1s 4ms/step - loss: 1.1365 - accuracy: 0.6421
test loss: 1.1365267038345337
test accuracy: 0.6420999765396118


In [ ]:
my_model.save("animals_detection_model_6625.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!wget -O test_cifar_gemi.jpg https://khosann.com/wp-content/uploads/2017/12/yük_gemisi-elektrikli-elektrikli_gemi-çin-guangzhou.jpg
!wget -O test_cifar_ucak.jpg https://www.airlineratings.com/wp-content/uploads/uploads/B797.jpg

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
my_model = load_model("animals_detection_model_6625.h5")

In [ ]:
image = load_img("att.webp", target_size=(32,32))
image = img_to_array(image) / 255
image = np.expand_dims(image, axis=0)
print("shape:",image.shape)

shape: (1, 32, 32, 3)


In [ ]:
preds = my_model.predict(image)

1/1 [==============================] - 0s 18ms/step


In [ ]:
preds

array([[6.40180311e-04, 6.81096688e-04, 9.83911008e-02, 3.43790352e-01,
        1.21408515e-01, 2.25280017e-01, 1.66904852e-01, 3.87216695e-02,
        1.10961417e-04, 4.07124404e-03]], dtype=float32)

In [ ]:
preds.argmax()

3

In [ ]:
classes[3]

'cat'

In [ ]:
# imagedatagenerator'ı oluşturucaz, kurallar belirlicez ve bu kurallar nezdinde de resimlerimiz çoklanacak ve bu  sayede daha fazla veriye erişmiş olacağız.